input_test, lowest location from range 46

In [1]:
import numpy as np
import re

In [2]:
#with open("input_test.txt", 'r') as f0:
with open("input.txt", 'r') as f0:
    text= f0.read()

In [3]:
class alm_map():
    def __init__(self, text_map):
        # text_map contains the text with lines providing map
        self.params_list=[]
        lines_map = text_map.split('\n')
        #print(lines_map)
        for l0 in lines_map:
            values_str = l0.split(' ')
            if len(values_str)!=3:
                #print(f"Error with this line {l0}, does not have 3 values")
                continue
            dest_orig = int(values_str[0])
            source_orig = int(values_str[1])
            length = int(values_str[2])
            self.params_list.append((dest_orig, source_orig, length))
    
    def source_to_dest(self,i):
        ret=i # default
        # go though list and finds the map
        for d0, s0, l0 in self.params_list:
            if i>=s0 and i<s0+l0:
                ret=d0+(i-s0)
                break
        return ret
    
    @staticmethod
    def _range_to_map_range(i0, i1, r0, r1, d):
        # put start end values in a list, convert to set to remove duplicates
        #range start, length i0,il. mapped range start, length r0 rl, destination start d
        
        start0 = max(i0,r0)
        end0 = min(i1,r1)

        ret_ranges_unmapped = [] #list of start,end
        ret_ranges_mapped = [] #list of start,end

        #Check region is inside the range and map it
        if start0<end0:
            ret_ranges_mapped.append((d+(start0-r0), d+(end0-r0)))

            #sort surrounding regions if any and mark them as unmmaped
            if start0>i0:
                ret_ranges_unmapped.append((i0,start0)) #start,end
            if i1>end0:
                ret_ranges_unmapped.append((end0,i1)) #start,end
        else:
            ret_ranges_unmapped.append((i0,i1))
            
        return [ret_ranges_unmapped,ret_ranges_mapped]
    
    def sources_to_dests_ranges(self, sources0):
        # use notation start,end
        #sources are a list of 2-tuples start:end
        unmapped_ranges0=sources0.copy() # initialize
        new_ranges=[]

        while len(unmapped_ranges0)>0:
            s0, s1 = unmapped_ranges0.pop()
            #print(f"origin range {s0}:{s1}")
            #Check against the mapping parameters
            new_unmapped = []
            new_mapped = []
            for d0, r0, l0 in self.params_list:
                r1=r0+l0 #enpoint
                #print(f"mapping r0:r1 {r0}:{r1} to d0:{d0}:{d0+l0}, be applied to unmapped_range:{s0}:{s1}")

                unmapped_ranges1, mapped_ranges1= self._range_to_map_range(s0,s1,r0,r1,d0)
                #print(f"result unmapped:{unmapped_ranges1}, mapped:{mapped_ranges1}")
                
                if len(mapped_ranges1)>0:
                    #print("len(mapped_ranges1)>0")
                    new_mapped+=mapped_ranges1 #mapped ranges increase
                    new_unmapped+=unmapped_ranges1
                    #Cannot use the original unmapped region, as it has alrady been transformed
                    # So leave the for-loop to get the next unmasked range
                    break

            # After checking an unmapped region against all the maps...
            # If there was no mapping applied, preserve it for next
            # If there was some mapping, then new_unmapped should contain the new maps
            # We need to check these new regions against the mappings
            if len(new_mapped)==0:
                #original range not affected
                #print(f"Region {s0}:{s1} not affected. new ranges+={(s0,s1)}")
                new_ranges+=[(s0,s1)]
            else:
                unmapped_ranges0+=new_unmapped
                new_ranges+=new_mapped
                #new_ranges+=new_unmapped

        return new_ranges

# Test _range_to_map_range

In [4]:
#Returns unmapped, mapped ranges, each in format start,length
print( alm_map._range_to_map_range(10,20, 5,15, 20) )

[[(15, 20)], [(25, 30)]]


Get starting seeds

In [5]:
seeds_inp = re.findall("seeds:(.*)", text)
seeds_inp

[' 79 14 55 13']

convert to seed ranges

In [6]:
seeds_inp[0].strip().split()

['79', '14', '55', '13']

In [7]:
seeds_ranges =[]

s_list = seeds_inp[0].strip().split()
for i in range(0,len(s_list),2):
    s0=int(s_list[i])
    s1=int(s_list[i+1]) +s0
    seeds_ranges.append((s0,s1))
seeds_ranges

[(79, 93), (55, 68)]

In [8]:
# Overrides for testing
#seeds_ranges=[(82,1)] #test ok, gives right answer 46

In [9]:
# seed2soil
t0= re.findall("seed-to-soil map:\n((.|\n)*)\nsoil-to-fertilizer map", text)
t0_map = alm_map(t0[0][0])

In [10]:
start_ranges = seeds_ranges.copy()

result_ranges = t0_map.sources_to_dests_ranges(start_ranges)
print(f"new_range:{result_ranges}")

soil0_ranges =result_ranges

new_range:[(57, 70), (81, 95)]


In [11]:
soil0_ranges

[(57, 70), (81, 95)]

soil-to-fertilizer map:

In [12]:
# soil2fert
t0 = re.findall("soil-to-fertilizer map:\n((.|\n)*)\nfertilizer-to-water map:", text)
t0_map = alm_map(t0[0][0])

# start_ranges = soil0_ranges
# mapped_ranges =[]

# for r0,rl in start_ranges:
#     #print(f"soil_range, {r0} , length:{rl}")
#     new_range = t0_map.source_to_dest_ranges(r0,rl)
#     #print(f"r0:{r0}, r1:{rl}, new_range:{new_range}")
#     mapped_ranges+=new_range

# fert0_ranges=mapped_ranges

start_ranges = soil0_ranges.copy() #Adjust

result_ranges = t0_map.sources_to_dests_ranges(start_ranges)
print(f"new_range:{result_ranges}")
    
fert0_ranges =result_ranges #Adjust

new_range:[(81, 95), (57, 70)]


In [13]:
fert0_ranges

[(81, 95), (57, 70)]

In [14]:
# fert2water
t0 = re.findall("fertilizer-to-water map:\n((.|\n)*)\nwater-to-light map", text)
t0_map = alm_map(t0[0][0])

# start_ranges = fert0_ranges
# mapped_ranges =[]

# for r0,rl in start_ranges:
#     #print(f"start_range, {r0} , length:{rl}")
#     new_range = t0_map.source_to_dest_ranges(r0,rl)
#     #print(f"r0:{r0}, r1:{rl}, new_range:{new_range}")
#     mapped_ranges+=new_range

# water0_ranges=mapped_ranges

start_ranges = fert0_ranges.copy() #Adjust

result_ranges = t0_map.sources_to_dests_ranges(start_ranges)
print(f"new_range:{result_ranges}")

water0_ranges =result_ranges #Adjust

new_range:[(53, 57), (61, 70), (81, 95)]


In [15]:
water0_ranges

[(53, 57), (61, 70), (81, 95)]

In [16]:
# water2light
t0 = re.findall("water-to-light map:\n((.|\n)*)\nlight-to-temperature map", text)
t0_map = alm_map(t0[0][0])

# start_ranges = water0_ranges
# mapped_ranges =[]

# for r0,rl in start_ranges:
#     #print(f"start_range, {r0} , length:{rl}")
#     new_range = t0_map.source_to_dest_ranges(r0,rl)
#     #print(f"r0:{r0}, r1:{rl}, new_range:{new_range}")
#     mapped_ranges+=new_range
# light0_ranges=mapped_ranges

start_ranges = water0_ranges.copy() #Adjust

result_ranges = t0_map.sources_to_dests_ranges(start_ranges)
print(f"new_range:{result_ranges}")

light0_ranges =result_ranges #Adjust

new_range:[(74, 88), (54, 63), (46, 50)]


In [17]:
light0_ranges

[(74, 88), (54, 63), (46, 50)]

In [18]:
# light2temp
t0 = re.findall("light-to-temperature map:\n((.|\n)*)\ntemperature-to-humidity map:", text)
t0_map = alm_map(t0[0][0])

# start_ranges = light0_ranges
# mapped_ranges =[]

# for r0,rl in start_ranges:
#     #print(f"start_range, {r0} , length:{rl}")
#     new_range = t0_map.source_to_dest_ranges(r0,rl)
#     #print(f"r0:{r0}, r1:{rl}, new_range:{new_range}")
#     mapped_ranges+=new_range
# temp0_ranges=mapped_ranges
# temp0_ranges

start_ranges = light0_ranges.copy() #Adjust

result_ranges = t0_map.sources_to_dests_ranges(start_ranges)
print(f"new_range:{result_ranges}")

temp0_ranges =result_ranges #Adjust

new_range:[(82, 86), (90, 99), (45, 56), (78, 81)]


In [19]:
temp0_ranges

[(82, 86), (90, 99), (45, 56), (78, 81)]

In [20]:
# temp2humid
t0 = re.findall("temperature-to-humidity map:\n((.|\n)*)\nhumidity-to-location map:", text)
t0_map = alm_map(t0[0][0])

# start_ranges = temp0_ranges
# mapped_ranges =[]

# for r0,rl in start_ranges:
#     #print(f"start_range, {r0} , length:{rl}")
#     new_range = t0_map.source_to_dest_ranges(r0,rl)
#     #print(f"r0:{r0}, r1:{rl}, new_range:{new_range}")
#     mapped_ranges+=new_range
# humid0_ranges=mapped_ranges
# humid0_ranges

start_ranges = temp0_ranges.copy() #Adjust

result_ranges = t0_map.sources_to_dests_ranges(start_ranges)
print(f"new_range:{result_ranges}")

humid0_ranges =result_ranges #Adjust

new_range:[(78, 81), (46, 57), (90, 99), (82, 86)]


In [21]:
humid0_ranges

[(78, 81), (46, 57), (90, 99), (82, 86)]

In [22]:
# hum2loc
t0 = re.findall("humidity-to-location map:\n((.|\n)*)\n", text)
t0_map = alm_map(t0[0][0])

# start_ranges = humid0_ranges
# mapped_ranges =[]

# for r0,rl in start_ranges:
#     #print(f"start_range, {r0} , length:{rl}")
#     new_range = t0_map.source_to_dest_ranges(r0,rl)
#     #print(f"r0:{r0}, r1:{rl}, new_range:{new_range}")
#     mapped_ranges+=new_range
# loc0_ranges=mapped_ranges
# loc0_ranges

start_ranges = humid0_ranges.copy() #Adjust

result_ranges = t0_map.sources_to_dests_ranges(start_ranges)
print(f"new_range:{result_ranges}")

loc0_ranges =result_ranges #Adjust

new_range:[(86, 90), (94, 97), (56, 60), (97, 99), (60, 61), (46, 56), (82, 85)]


In [23]:
loc0_ranges

[(86, 90), (94, 97), (56, 60), (97, 99), (60, 61), (46, 56), (82, 85)]

In [24]:
# lowest number
loc_starts=[]
for loc0 in loc0_ranges:
    s0,l0 = loc0
    loc_starts.append(s0)

print(np.min(loc_starts))

46


Answer 149145352 ? is too high!!

Correct answer is 37384986